In [5]:
import pandas as pd
import datetime as dt
import os

def parse_feed_info_file(feed_info_file_path):
    feed_info_df = pd.read_csv(feed_info_file_path)
    # save start and end date of feed and convert them to datetime
    feed_info_df['feed_start_date'] = pd.to_datetime(feed_info_df['feed_start_date'], format='%Y%m%d')
    feed_info_df['feed_end_date'] = pd.to_datetime(feed_info_df['feed_end_date'], format='%Y%m%d')
    return feed_info_df.iloc[0]['feed_start_date'], feed_info_df.iloc[0]['feed_end_date']

def parse_calendar_file(calendar_file_path):
    calendar_df = pd.read_csv(calendar_file_path)
    # Convert start_date and end_date to datetime objects
    calendar_df['start_date'] = pd.to_datetime(calendar_df['start_date'], format='%Y%m%d')
    calendar_df['end_date'] = pd.to_datetime(calendar_df['end_date'], format='%Y%m%d')

    calendar_data = {}
    for index, row in calendar_df.iterrows():
        service_id = row['service_id']
        start_date = row['start_date']
        end_date = row['end_date']
        days = row[['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']]
        for date in pd.date_range(start_date, end_date):
            if date not in calendar_data:
                calendar_data[date] = set()
            if days.iloc[date.dayofweek] == 1:
                calendar_data[date].add(service_id)
    return calendar_data

def parse_calendar_dates_file(calendar_dates_file_path):
    calendar_dates_df = pd.read_csv(calendar_dates_file_path)
    # Convert date to datetime object
    calendar_dates_df['date'] = pd.to_datetime(calendar_dates_df['date'], format='%Y%m%d')
    calendar_dates_data = {}
    for index, row in calendar_dates_df.iterrows():
        service_id = row['service_id']
        date = row['date']
        exception_type = row['exception_type']
        if date not in calendar_dates_data:
            calendar_dates_data[date] = set()
        if exception_type == 1:
            calendar_dates_data[date].add(service_id)
        elif exception_type == 2:
            if service_id in calendar_dates_data[date]:
                calendar_dates_data[date].remove(service_id)
    return calendar_dates_data

def generate_schedule(feed_start_date, feed_end_date, calendar_data, calendar_dates_data):
    schedule = []
    for date in pd.date_range(feed_start_date, feed_end_date):
        day_of_week = date.strftime('%A')
        service_ids = calendar_data.get(date, set()) | calendar_dates_data.get(date, set())
        schedule.append((date, day_of_week, service_ids))
    return schedule

In [2]:
working_path = 'C:\\Users\\antonio.forte\\Dropbox (MIT)\\GitHub\\getGTFS-RT\\MBTA_PostRatingRecap_ArrDepTimes'
calendar_file_path = os.path.join(working_path, 'gtfsSchedule','gtfs_2022-12-18_2023-03-11_Winter2023PostRecap','calendar.txt')
calendar_dates_file_path = os.path.join(working_path, 'gtfsSchedule','gtfs_2022-12-18_2023-03-11_Winter2023PostRecap','calendar_dates.txt')
feed_info_file_path = os.path.join(working_path, 'gtfsSchedule','gtfs_2022-12-18_2023-03-11_Winter2023PostRecap','feed_info.txt')

feed_start_date, feed_end_date = parse_feed_info_file(feed_info_file_path)
calendar_data = parse_calendar_file(calendar_file_path)
calendar_dates_data = parse_calendar_dates_file(calendar_dates_file_path)
schedule = generate_schedule(feed_start_date, feed_end_date, calendar_data, calendar_dates_data)

for entry in schedule:
    print(entry)

(Timestamp('2022-12-18 00:00:00'), 'Sunday', {'RTL123-3-Su-01', 'LRV123-7-Su-01', 'RTL123-F-Su-01-AstJFK', 'FLL 22-N-Su-1-S72', 'FLL 22-S-Su-1-S86', 'FLL 22-S-Su-1-Sb4', 'FLL 22-S-Su-1', 'CR-2-LynnShuttleFll-Snd', 'FLL 22-S-Su-1-S8f', 'FLL 22-N-Su-5', 'FLL 22-N-Su-1', 'PRIV123-5-Su-01', 'FLL 22-N-Su-1-S6d', 'FLL 22-S-Su-1-Sc0', 'FLL 22-S-Su-1-Sc5', 'RTL123-5-Su-01', 'FLL 22-S-Su-1-Sb6'})
(Timestamp('2022-12-19 00:00:00'), 'Monday', {'FLL 22-N-Wdy-1', 'PRIV123-1-Wdy-01', 'RTL123-D-Wdy-01', 'RTL123-A-Wdy-01', 'Fall/WinterWeekday', 'LRV123-5-Wdy-01', 'Boat-F4-Wdy-Fll-22', 'LRV123-1-Wdy-01', 'FLL 22-N-Wdy-1-S6d', 'PRIV123-3-Wdy-01', 'FLL 22-S-Wdy-1-Sc5', 'FLL 22-S-Wdy-1-S86', 'Boat-F1-Wdy-Fll-22', 'RTL123-1-Wdy-01', 'CR-2-LynnShuttleWtr-Wkd'})
(Timestamp('2022-12-20 00:00:00'), 'Tuesday', {'FLL 22-N-Wdy-1', 'PRIV123-1-Wdy-01', 'RTL123-D-Wdy-01', 'RTL123-A-Wdy-01', 'Fall/WinterWeekday', 'LRV123-5-Wdy-01', 'Boat-F4-Wdy-Fll-22', 'LRV123-1-Wdy-01', 'FLL 22-N-Wdy-1-S6d', 'PRIV123-3-Wdy-01', 'FL

C:\Users\antonio.forte\AppData\Local\Temp\ipykernel_27608\3915915932.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if days[date.dayofweek] == 1:
C:\Users\antonio.forte\AppData\Local\Temp\ipykernel_27608\3915915932.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if days[date.dayofweek] == 1:
C:\Users\antonio.forte\AppData\Local\Temp\ipykernel_27608\3915915932.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if days[da